In [4]:
import tkinter as tk
from tkinter import messagebox
import json
import os

# ---------------- File Setup ----------------
USER_FILE = "users.json"

def load_users():
    """Load users and their payment data from JSON file."""
    if os.path.exists(USER_FILE):
        try:
            with open(USER_FILE, "r") as f:
                return json.load(f)
        except json.JSONDecodeError:
            return {}
    return {}

def save_users(users):
    """Save users and their payment data to JSON file."""
    with open(USER_FILE, "w") as f:
        json.dump(users, f, indent=4)

# ---------------- Registration ----------------
def register():
    users = load_users()
    first = first_name_entry.get().strip()
    last = last_name_entry.get().strip()
    email = email_entry.get().strip()
    pwd = password_entry.get()
    confirm_pwd = confirm_password_entry.get()

    if not first or not last or not email or not pwd or not confirm_pwd:
        messagebox.showerror("Error", "All fields are required")
        return
    if pwd != confirm_pwd:
        messagebox.showerror("Error", "Passwords do not match")
        return
    if email in users:
        messagebox.showerror("Error", "Email already registered")
        return

    users[email] = {
        "first_name": first,
        "last_name": last,
        "password": pwd,
        "payments": {}  # initialize payment record
    }
    save_users(users)
    messagebox.showinfo("Success", f"Account for {first} {last} created successfully!")
    clear_entries()

# ---------------- Login ----------------
current_user = None  # store logged-in user email

def login():
    global current_user
    users = load_users()
    email = login_email_entry.get().strip()
    pwd = login_password_entry.get()

    if email in users and users[email]["password"] == pwd:
        current_user = email
        show_payment_frame()
        update_status_list()
    else:
        messagebox.showerror("Login Failed", "Invalid email or password")

# ---------------- Entry Clear Functions ----------------
def clear_entries():
    first_name_entry.delete(0, tk.END)
    last_name_entry.delete(0, tk.END)
    email_entry.delete(0, tk.END)
    password_entry.delete(0, tk.END)
    confirm_password_entry.delete(0, tk.END)

def clear_login_entries():
    login_email_entry.delete(0, tk.END)
    login_password_entry.delete(0, tk.END)

# ---------------- Payment Functions ----------------
def make_payment():
    month = month_var.get()
    year = year_var.get()
    code = code_entry.get().strip()

    if not month or not year or not code:
        messagebox.showerror("Error", "All fields are required")
        return

    # Validate payment code
    if code != "*182*8*1*355613":
        messagebox.showerror("Error", "Invalid payment code")
        return

    users = load_users()
    if current_user not in users:
        messagebox.showerror("Error", "User not found")
        return

    # Mark the month-year as Paid
    users[current_user]["payments"][f"{month}-{year}"] = "Paid"
    save_users(users)
    messagebox.showinfo("Success", f"Payment for {month}-{year} recorded successfully!")
    code_entry.delete(0, tk.END)
    update_status_list()

def update_status_list():
    status_list.delete(0, tk.END)
    users = load_users()
    payments = users[current_user].get("payments", {})
    months = ["January","February","March","April","May","June",
              "July","August","September","October","November","December"]
    years = ["2025","2026","2027"]
    for y in years:
        for m in months:
            key = f"{m}-{y}"
            status = payments.get(key, "Pending")
            status_list.insert(tk.END, f"{key}: {status}")

# ---------------- GUI ----------------
root = tk.Tk()
root.title("UMUSANZU - Payment System")
root.geometry("450x600")
root.resizable(False, False)

# --- Switch Frame ---
switch_frame = tk.Frame(root)
switch_frame.pack(pady=10)

def show_register_frame():
    login_frame.pack_forget()
    payment_frame.pack_forget()
    register_frame.pack()

def show_login_frame():
    register_frame.pack_forget()
    payment_frame.pack_forget()
    login_frame.pack()

tk.Button(switch_frame, text="Register", width=15, command=show_register_frame).grid(row=0, column=0, padx=10)
tk.Button(switch_frame, text="Login", width=15, command=show_login_frame).grid(row=0, column=1, padx=10)

# --- Registration Frame ---
register_frame = tk.Frame(root)

tk.Label(register_frame, text="First Name").pack(pady=5)
first_name_entry = tk.Entry(register_frame)
first_name_entry.pack()

tk.Label(register_frame, text="Last Name").pack(pady=5)
last_name_entry = tk.Entry(register_frame)
last_name_entry.pack()

tk.Label(register_frame, text="Email").pack(pady=5)
email_entry = tk.Entry(register_frame)
email_entry.pack()

tk.Label(register_frame, text="Password").pack(pady=5)
password_entry = tk.Entry(register_frame, show="*")
password_entry.pack()

tk.Label(register_frame, text="Confirm Password").pack(pady=5)
confirm_password_entry = tk.Entry(register_frame, show="*")
confirm_password_entry.pack()

tk.Button(register_frame, text="Register", width=20, command=register).pack(pady=10)

# --- Login Frame ---
login_frame = tk.Frame(root)

tk.Label(login_frame, text="Email").pack(pady=5)
login_email_entry = tk.Entry(login_frame)
login_email_entry.pack()

tk.Label(login_frame, text="Password").pack(pady=5)
login_password_entry = tk.Entry(login_frame, show="*")
login_password_entry.pack()

tk.Button(login_frame, text="Login", width=20, command=login).pack(pady=10)

# --- Payment Frame ---
payment_frame = tk.Frame(root)

tk.Label(payment_frame, text="Select Month").pack(pady=5)
month_var = tk.StringVar()
month_menu = tk.OptionMenu(payment_frame, month_var, "January","February","March","April","May","June",
                           "July","August","September","October","November","December")
month_menu.pack()

tk.Label(payment_frame, text="Select Year").pack(pady=5)
year_var = tk.StringVar()
year_menu = tk.OptionMenu(payment_frame, year_var, "2025","2026","2027")
year_menu.pack()

tk.Label(payment_frame, text="Payment Code").pack(pady=5)
code_entry = tk.Entry(payment_frame)
code_entry.pack()

tk.Button(payment_frame, text="Pay", width=20, command=make_payment).pack(pady=10)

tk.Label(payment_frame, text="Payment Status").pack(pady=5)
status_list = tk.Listbox(payment_frame, width=40)
status_list.pack(pady=5)

tk.Button(payment_frame, text="Refresh Status", width=20, command=update_status_list).pack(pady=10)

# Show registration frame by default
register_frame.pack()

# Run the application
root.mainloop()
